# Deep CNN-dan foydalangan holda MNIST qo'lda yozilgan raqamlarni aniqlovchi
![MNIST Dataset](https://miro.medium.com/max/1400/1*26W2Yk3cu2uz_R8BuSb_SA.png)
   Agar siz Mashinani o'rganish va ma'lumotlar faniga qiziqqan bo'lsangiz va siz mashinani o'rganish tushunchalarini o'rganish yo'lini izlayotgan bo'lsangiz, siz o'rganish va tarbiyalanish uchun eng zo'r joyga keldingiz. Kaggle yangi boshlanuvchilar uchun mashinani o'rganishni boshlash uchun juda mos keladi. MNIST ma'lumotlar to'plamidan foydalangan holda qo'lda yozilgan raqamni aniqlovchi barcha yangi boshlanuvchilar uchun musobaqani boshlaydi. Umid qilamanki, mening daftarim sizni mashinani o'rganish dunyosi va kelajak bilan tanishtirishi mumkin.

   Ushbu yadroda men turli xil qo'lda yozilgan raqamlarni tanib olish va ularni tasniflash uchun chuqur konvolyutsion neyron tarmog'i (CNN) modelini yaratdim. Bu erda foydalanilgan ma'lumotlar to'plami aslida [Raqamlarni aniqlash tanlovi](https://www.kaggle.com/c/digit-recognizer) dan olingan.

**Frameworks**
* [Tensorflow v2](https://tensorflow.org) - Google’ning ochiq manbali mashina oʻrganish tizimi.
* [Keras](https://keras.io) - Tensorflow ustida ishlaydigan ochiq manbali neyron tarmoq kutubxonasi.


### Jarayon
1. [Kutubxonalarni import qilish](#1)
2. [Ma'lumotlar to'plamini tayyorlash](#2)
3. [Model qurilishi](#3)
4. [Modelni o'rnatish](#4)
5. [Modelni tahlil qilish](#5)
6. [Sinov ma'lumotlarini bashorat qilish](#6)

## 1. Kutubxonalarni import qilish <a></a>
Bu erda biz yadro uchun kerakli kutubxonalarni import qilamiz.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools


sns.set(style="dark",context="notebook",palette="muted")

#Tensorflow versiyalari
print("TensorFlow versiyasi:   "+tf.version.VERSION)
print("Keras versiyasi:   "+tf.keras.__version__)

## 2. Ma'lumotlar to'plamini tayyorlash
Biz MNIST Handwritten Digit Recognition tanlovidan olingan ma'lumotlarni qo'shdik. Buning uchun yuqori o'ng burchakdagi *+ Ma'lumot qo'shish* tugmasidan foydalaning. Raqobatni tanlang. Ma'lumotlar yadroga qo'shiladi. Ma'lumotlar to'plamini o'ng panelda Ma'lumotlar ostida topishimiz mumkin. Siz poezdning yo'lini nusxalashingiz va o'ng paneldan ma'lumotlarni sinab ko'rishingiz mumkin.

read_csv csv faylini konvertatsiya qilishdan pandas DataFrame obyektini qaytarish uchun ishlatiladi.

Keyin biz yorliq ustunini tanlaymiz va uni trening uchun ishlatiladigan Y_train da saqlaymiz. X_train tegishli etiketli tasvirning piksel qiymatlarini o'z ichiga oladi.

Biz har bir sinf ma'lumotlarining umumiy sonini countplot yordamida tasavvur qilamiz. Keyin ma'lumotlar to'plamida etishmayotgan qiymatlarni tekshiramiz.

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
Y_train = train['label']

#Target ustunini olib tashlaymiz
X_train = train.drop(labels=['label'],axis=1)

#Keraksiz o'zgaruvchidagi egallab turgan joyini bo'shatamiz
del train

graph = sns.countplot(Y_train)
 
Y_train.value_counts()

In [ ]:
#Outlier data va tushib qolgan ma'lumotlarni tekshiramiz
X_train.isnull().any().describe()

test.isnull().any().describe()


### Normalizatsiya
Kirish qiymatlari shkalasini kamaytirish uchun normallashtirish amalga oshiriladi. Piksel qiymati 0 dan 255 gacha, bu kul rang gradientini belgilaydi. CNN 0 dan 1 gacha bo'lgan qiymatlarda 0 dan 255 gacha tezroq birlashadi. Shunday qilib, biz ma'lumotlarni [0..255] dan [0..1] gacha masshtablash uchun har bir qiymatni 255 ga bo'lamiz. Agar bizda kattaroq ma'lumotlarga ega bo'lsak, u hisoblash murakkabligini kamaytirish orqali modelga xususiyatlarni yaxshiroq o'rganishga yordam beradi.

In [ ]:
X_train = X_train/255
test = test/255

### Shaklni o'zgartirish
Piksel qiymatlari massivi (28,28,1) matritsaga o'zgartiriladi. Biz CNN modelini 28x28x1 matritsali input_shape bilan oziqlantirmoqdamiz.

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

### Targetni kodlash
Chunki CNN modeli har bir sinf uchun bashorat vektorida natijalar beradi. Yorliq (raqamlar) model tomonidan bashorat qilish uchun issiq vektorga kodlangan. Shunday qilib, biz CNNni kodlangan chiqishlar bilan o'rgatishimiz mumkin va parametrlar mos ravishda sozlanadi

In [ ]:
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=10)

### Poezd va tasdiqlash ma'lumotlarini ajratish
Biz o'qitish uchun kiritilgan ma'lumotlarni ikkita eksklyuziv ma'lumotlarga ajratamiz, xususan, Train va Validation ma'lumotlar to'plami. Poezd ma'lumotlari modelni o'qitish uchun ishlatiladi, tasdiqlash ma'lumotlari esa modelning to'g'riligini va modelning o'quv ma'lumotlaridan tashqari ma'lumotlar uchun qanchalik yaxshi umumlashtirilganligini o'zaro tekshirish uchun ishlatiladi. Tasdiqlashning aniqligi va yo'qolishi bizga yangi ma'lumotlar uchun modelning ishlashini aytib beradi va modelni o'rgatish paytida haddan tashqari mos yoki noto'g'ri vaziyat mavjudligini ko'rsatadi.


In [ ]:
random_seed =2

X_train,X_val,Y_train,Y_val = train_test_split(X_train,Y_train,test_size=0.1,
                                                random_state = random_seed)

In [ ]:
g = plt.imshow(X_train[0][:,:,0])

## 3. Model Arxitekturasi

Chuqur konvolyutsion neyron tarmog'i - bu sun'iy neyron tarmoqlar tarmog'i. Model arxitekturasi - bu biz o'qitish jarayonida parametrlarni o'rgatadigan neyron tarmoqlarning dizayni. Bu yerda biz LeNet-5 arxitekturasidan foydalandik, u 1998 yilda Yann LeKun tomonidan taklif qilingan. U oʻzining minimal tuzilishi va oʻrgatish oson tabiati bilan juda mashhur. LeNet-5 arxitekturasi kichik o'lchamdagi tasvirlarda turli sinf ob'ektlarini tanib olish va tasniflash uchun javob beradi.

CNN arxitekturalari va zamonaviy arxitekturasi haqida ko'proq ma'lumot olish uchun [bu yerga](https://medium.com/analytics-vidhya/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5) tashrif buyuring.

![Deep CNN Architecture](https://miro.medium.com/max/3744/1*SGPGG7oeSvVlV5sOSQ2iZw.png)

In [ ]:
model = tf.keras.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=(5,5), padding='Same', 
                       activation=tf.nn.relu, input_shape = (28,28,1)))
model.add(layers.Conv2D(filters=32, kernel_size=(5,5), padding='Same', 
                       activation=tf.nn.relu))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same', 
                       activation=tf.nn.relu, input_shape = (28,28,1)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same', 
                       activation=tf.nn.relu))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256,activation=tf.nn.relu))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10,activation=tf.nn.softmax))

### Optimizatorlar va yumshatgichlar
* Optimizator neyron tarmoqning hal qiluvchi qismidir. Optimizator modelning optimalga tezroq yetib borishini ta'minlaydi. RMSProp optimallashtiruvchisi modelni yanada samarali va tezroq birlashtirishga imkon beradi. Shuningdek, u modelni global minimal darajada qamrab olishni qat'iy talab qiladi, shuning uchun modelning aniqligi yuqori bo'ladi.
* O'rganish tezligini o'rnatish Deep Learning algoritmida juda muhimdir. Yaxshi o'rganish tezligini tanlash o'z samarasini berishi mumkin bo'lsa-da, o'rganish tezligini pasaytirishni rejalashtirish global minimal konvergentsiyada katta afzalliklarga ega. ReduceLRonPlateau o'rganish tezligi va davrini kuzatish orqali o'rganish tezligini pasaytirishga imkon beradi. Bu modelga maksimal aniqlikka erishish uchun katta yordam beradi.

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)


In [ ]:
model.compile(optimizer = optimizer, loss='categorical_crossentropy', 
             metrics=["accuracy"])

In [ ]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc',
                                           patience=3,
                                           verbose=1,
                                           factor=0.5,
                                           min_lr=0.00001)

> Men kernel uchun epochni 10 ga qo'ydim. Kattaroq aniqlikni ko'rish uchun uni 30 ga oshirishingiz mumkin.

In [ ]:
epochs=30
batch_size = 112

### Ma'lumotlarni ko'paytirish
Ma'lumotlarni ko'paytirish - bu berilgan tasvirlarni manipulyatsiya qilish orqali o'qitish uchun qo'shimcha ma'lumotlar to'plamini yaratish jarayoni. Chuqur o'rganishda katta ma'lumotlar to'plamining mavjudligi trening uchun juda muhimdir. Bizda haqiqiy dunyo o'quv namunalari cheklanganligi sababli, biz mashg'ulotlar uchun ko'proq tasvirlar yaratish uchun ma'lumotlarni kengaytirishdan foydalanishimiz mumkin. Ma'lumotlarni ko'paytirish o'qitish uchun qo'shimcha ma'lumotlarni yaratish uchun mavjud ma'lumotlar to'plami bo'ylab kattalashtirish, aylantirish, aylantirish, kesish va boshqa tasvirlarni manipulyatsiya qilishni o'z ichiga oladi. Ma'lumotlarni ko'paytirish modelni umumiyroq tasniflashga imkon beradi.

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,  
        zca_whitening=False, 
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False)  


datagen.fit(X_train)

## 4. Modelni o'rnatish
   Modelni o'rnatish yoki Modelni o'rgatish - bu erda biz modelimizni o'rgatamiz va xato parametrlarini baholaymiz. O'quv jarayoni odatda protsessorda ishlaganda ko'p vaqt talab etadi. Ammo mashg'ulotlarni CUDA qo'llab-quvvatlaydigan grafik kartalar yordamida tezlashtirish mumkin. Kaggle o'rnatilgan cheklangan GPU qo'llab-quvvatlashiga ega, bu hujayrani ishga tushirayotganda uni yoqishni unutmang.

In [ ]:
if(tf.test.is_built_with_cuda() == True):
    print("CUDA topildi")
else: 
    print("CUDA topilmadi")

In [ ]:
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

## 5. Modelni tahlil qilish
Biz modelni turli usullar yordamida tahlil qilishimiz mumkin. Ulardan biri grafikni o'rganishdir. Bu erda biz o'qitish va tekshirish ma'lumotlarining yo'qotishlarini chizmada tuzamiz va ularni trendga qarab baholaymiz. Ideal model uchun o'qitish va tekshirish yo'qotilishi past va shunga o'xshash bo'lishi kerak.

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="O'rganish xatosi")
ax[0].plot(history.history['val_loss'], color='r', label="Test xatosi",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="O'rganish xatosi")
ax[1].plot(history.history['val_accuracy'], color='r',label="Test xatosi")
legend = ax[1].legend(loc='best', shadow=True)

### Confusion matritsasi chizmasi
Confusion matritsasi - bu modelni baholashning yana bir usuli. U modelning ishlashini grafik tasvirlash uchun ishlatiladi. U har bir sinfni bashorat qilishda Modelning ishlashini ko'rsatadi. Bu erda siz modelni tegishli sinflarni juda aniq bashorat qilishingiz mumkin.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matritsa',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel("To'g'ri qiymat")
    plt.xlabel('Model bashorati')


Y_pred = model.predict(X_val)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_val,axis = 1) 
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

### Muhim xato
Biz juda yuqori aniqlikka ega bo'lishimiz mumkin bo'lsa-da, har qanday chuqur o'rganish modeli barcha tasvirlarni to'g'ri bashorat qila olmaydi. Shunday qilib, biz istiqbolli modelimiz tomonidan qilingan muhim xatolarni ko'rib chiqamiz.

In [ ]:
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title(" Bashorat :{} To'g'ri :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)
most_important_errors = sorted_dela_errors[-6:]
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

## 6. Test ma'lumotlarini bashorat qilish
Nihoyat, biz mashg'ulotlar va ish faoliyatini baholashdan so'ng raqobat uchun test ma'lumotlar to'plamini bashorat qilmoqdamiz. Biz test ma'lumotlarini bashorat qilamiz va tanlovni topshirish uchun CSV faylida saqlaymiz.

In [ ]:
results = model.predict(test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

## Xulosa
Umid qilamanizki, barchangizga daftar yoqdi. Bu mening birinchi yozishim. Men Deep Learning va uning ortidagi matematik tushunchalar haqida ko'plab ajoyib yangi narsalarni bilib oldim. Men bu tushunchalarning barchasidan hayratda qoldim va chuqur o'rganish dunyosiga tushib qoldim.

## Agar siz ushbu daftarni foydali deb bilsangiz, iltimos, yuqori ovoz bering, bu meni kelajakda o'rganishga va ko'proq ishlashga undaydi.